In [161]:
import pandas as pd
import glob
import os
import numpy as np
from featureranker.utils import *
from featureranker.plots import *
from featureranker.rankers import *
from tqdm.auto import tqdm

xlsx_path = './betting_odds/'
txt_path = './game_data/'
paths = [path for path in glob.glob(xlsx_path + '*.xlsx')]
paths_txt = [path_txt for path_txt in glob.glob(txt_path + '*.txt')]
for path in paths:
    xlsx_dataframes = pd.read_excel(path)
    xlsx_dataframes.columns = xlsx_dataframes.columns.str.replace('[^a-zA-Z0-9]', '')
    view_data(xlsx_dataframes)


txt_dataframes = []
for path_txt in paths_txt:
    df = pd.read_csv(path_txt, delimiter = ",", quotechar='"', quoting=1, header = None)
    # df.columns = df.columns.astype(str).str.replace('[^a-zA-Z0-9]', '')
    txt_dataframes.append(df)
    view_data(df)
# txt_dataframes = txt_dataframes.replace({',': '', '\"': ''}, regex=True)

# If you want to view the file in Excel, uncomment the following line
# os.startfile(xlsx_files[0])


There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
The column 9th has 0.1% NaN values.
The column Pitcher has 0.6% NaN values.
The column 9th has 0.1% NaN values.
The column Pitcher has 0.2% NaN values.
The column 8th has 0.1% NaN values.
The column 9th has 0.1% NaN values.
There are no NaN values in the dataset
There are no NaN values in the dataset
There are no NaN values in the dataset
The column 13 has 99.9% NaN values.
The column 14 has 100.0% NaN values.
The column 15 has 99.9% NaN values.
The column 85 has 100.0% NaN values.
The column 87 has 100.0% NaN values.
The column 97 has 50.5% NaN values.
The column 99 has 5.1% NaN values.
The column 159 has 99.5% NaN values.
The column 13 has 100.0% NaN values.
The column 14 has 100.0% NaN values.
The column 15 has 100.0% NaN values.
The column 81 has 0.3% NaN va

In [162]:
def process_dataframes(paths):
    xlsx_dataframes = [pd.read_excel(path) for path in paths]
    merged_df = pd.concat(xlsx_dataframes)
    home_teams_df = merged_df[merged_df['VH'] == 'H']
    visiting_teams_df = merged_df[merged_df['VH'] == 'V']

    selected_columns_df = pd.DataFrame()
    for i in range(len(xlsx_dataframes)):
        year = 2010 + i
        temp_df = pd.DataFrame({
            'date': home_teams_df['Date'].apply(lambda x: str(year) + str(x).zfill(4)).reset_index(drop=True),
            'home_team': home_teams_df['Team'].reset_index(drop=True),
            'visiting_team': visiting_teams_df['Team'].reset_index(drop=True),
            'home_open': home_teams_df['Open'].reset_index(drop=True),
            'visiting_open': visiting_teams_df['Open'].reset_index(drop=True)
        })
        selected_columns_df = pd.concat([selected_columns_df, temp_df])

    return selected_columns_df

print(process_dataframes(paths))


           date home_team visiting_team home_open visiting_open
0      20100404       BOS           NYY      -114          -106
1      20100405       WAS           PHI       170          -200
2      20100405       NYM           MIA      -115          -105
3      20100405       CIN           STL       135          -155
4      20100405       PIT           LOS       135          -155
...         ...       ...           ...       ...           ...
28001  20211027       HOU           ATL      -115          -105
28002  20211029       ATL           HOU      -115          -105
28003  20211030       ATL           HOU      -115          -105
28004  20211031       ATL           HOU      -105          -115
28005  20211102       HOU           ATL      -120           100

[336072 rows x 5 columns]


In [163]:
print(selected_columns_df.head(16))


        date home_team visiting_team home_open visiting_open
0   20100404       BOS           NYY      -114          -106
1   20100405       WAS           PHI       170          -200
2   20100405       NYM           MIA      -115          -105
3   20100405       CIN           STL       135          -155
4   20100405       PIT           LOS       135          -155
5   20100405       MIL           COL      -125           105
6   20100405       ATL           CUB      -130           110
7   20100405       ARI           SDG      -175           155
8   20100405       HOU           SFO       110          -130
9   20100405       CWS           CLE      -170           150
10  20100405       TEX           TOR      -135           115
11  20100405       KAN           DET      -115          -105
12  20100405       LAA           MIN      -135           115
13  20100405       OAK           SEA       105          -125
14  20100406       HOU           SFO      -145           125
15  20100406       MIL  

In [164]:
print(txt_dataframes[0])


           0    1    2    3   4    5    6   7    8    9    ...  \
0     20100404    0  Sun  NYA  AL    1  BOS  AL    1    7  ...   
1     20100405    0  Mon  MIN  AL    1  ANA  AL    1    3  ...   
2     20100405    0  Mon  CLE  AL    1  CHA  AL    1    0  ...   
3     20100405    0  Mon  DET  AL    1  KCA  AL    1    8  ...   
4     20100405    0  Mon  SEA  AL    1  OAK  AL    1    5  ...   
...        ...  ...  ...  ...  ..  ...  ...  ..  ...  ...  ...   
2425  20101003    0  Sun  CHN  NL  162  HOU  NL  162    0  ...   
2426  20101003    0  Sun  ARI  NL  162  LAN  NL  162    1  ...   
2427  20101003    0  Sun  WAS  NL  162  NYN  NL  162    2  ...   
2428  20101003    0  Sun  SDN  NL  162  SFN  NL  162    0  ...   
2429  20101003    0  Sun  COL  NL  162  SLN  NL  162    1  ...   

                      151  152       153             154  155       156  \
0               J.D. Drew    9  camem001    Mike Cameron    8  scutm001   
1          Howie Kendrick    4  woodb003    Brandon Wood 

In [165]:
def extract_data(txt_dataframes, selected_columns_df):
    extracted_data_df = pd.DataFrame()
    for txt_df in txt_dataframes:
        # Extracting required data from the columns based on the provided indices
        temp_df = pd.DataFrame({
            'date': txt_df.iloc[:, 0].apply(lambda x: str(x)),
            'home_team': txt_df.iloc[:, 6],
            'visiting_team': txt_df.iloc[:, 3],
            'home_pitcher': txt_df.iloc[:, 104],
            'visiting_pitcher': txt_df.iloc[:, 102],
            'home_score': txt_df.iloc[:, 10],  # Added home score from column 10
            'visiting_score': txt_df.iloc[:, 9]  # Added visiting score from column 9
        })
        # Extracting player names for home and visiting teams
        for i in range(9):
            temp_df['home_player_'+str(i+1)] = txt_df.iloc[:, 133+3*i].str.split(',').str[0].replace(np.nan, 'Unknown Player')
        for i in range(9):
            temp_df['visiting_player_'+str(i+1)] = txt_df.iloc[:, 106+3*i].str.split(',').str[0].replace(np.nan, 'Unknown Player')
        extracted_data_df = pd.concat([extracted_data_df, temp_df])

    # Merging the extracted data with the selected_columns_df on 'date', 'home_team', and 'visiting_team'
    final_df = pd.merge(extracted_data_df, selected_columns_df,  how='left', left_on=['date','home_team', 'visiting_team'], right_on = ['date','home_team', 'visiting_team'])

    return final_df

selected_columns_df = process_dataframes(paths)
final_df = extract_data(txt_dataframes, selected_columns_df)
print(final_df)








           date home_team visiting_team    home_pitcher  visiting_pitcher  \
0      20100404       BOS           NYA    Josh Beckett       CC Sabathia   
1      20100405       ANA           MIN    Jered Weaver       Scott Baker   
2      20100405       CHA           CLE    Mark Buehrle    Jake Westbrook   
3      20100405       KCA           DET    Zack Greinke  Justin Verlander   
4      20100405       OAK           SEA      Ben Sheets   Felix Hernandez   
...         ...       ...           ...             ...               ...   
33890  20221005       CLE           KCA    Aaron Civale       Jon Heasley   
33891  20221005       HOU           PHI  Framber Valdez     Bailey Falter   
33892  20221005       OAK           ANA   Ken Waldichuk     Shohei Ohtani   
33893  20221005       SEA           DET  Marco Gonzales   Tyler Alexander   
33894  20221005       TEX           NYA      Glenn Otto    Domingo German   

       home_score  visiting_score    home_player_1    home_player_2  \
0   

In [197]:
print(final_df.iloc[5])


date                           20100405
home_team                           OAK
visiting_team                       SEA
home_pitcher                 Ben Sheets
visiting_pitcher        Felix Hernandez
home_score                            3
visiting_score                        5
home_player_1               Rajai Davis
home_player_2              Daric Barton
home_player_3              Ryan Sweeney
home_player_4          Kevin Kouzmanoff
home_player_5               Kurt Suzuki
home_player_6               Eric Chavez
home_player_7                Mark Ellis
home_player_8               Travis Buck
home_player_9          Cliff Pennington
visiting_player_1         Ichiro Suzuki
visiting_player_2         Chone Figgins
visiting_player_3        Casey Kotchman
visiting_player_4        Milton Bradley
visiting_player_5           Ken Griffey
visiting_player_6            Jose Lopez
visiting_player_7    Franklin Gutierrez
visiting_player_8           Rob Johnson
visiting_player_9           Jack Wilson


In [167]:
final_df.to_csv('./game_and_player_data/final_df.csv', index=False)


In [210]:
class BettingWallet:
    def __init__(self, bet_amount, bet_team, bet_date, home_team, visiting_team):
        self.bet_amount = bet_amount
        self.bet_team = bet_team  # 1 for visiting team, 0 for home team
        self.bet_date = bet_date
        self.home_team = home_team
        self.visiting_team = visiting_team
        self.wallet_balance = 100.0

    def calculate_winner(self, row):
        if row['home_score'] > row['visiting_score']:
            return 0  # home team wins
        else:
            return 1  # visiting team wins

    def calculate_payout(self, row, winner):
        print(row['visiting_open'])
        if winner == 0:  # home team wins
            if row['home_open'] > 0:
                return self.bet_amount * (row['home_open'] / 100.0)
            else:
                return self.bet_amount * (100.0 / abs(row['home_open']))
        else:  # visiting team wins
            if row['visiting_open'] > 0:
                return self.bet_amount * (row['visiting_open'] / 100.0)
            else:
                return self.bet_amount * (100.0 / abs(row['visiting_open']))

    def place_bet(self, df):
        for index, row in df.iterrows():
            if row['date'] == self.bet_date and row['home_team'] == self.home_team and row['visiting_team'] == self.visiting_team:
                winner = self.calculate_winner(row)
                if winner == self.bet_team:
                    self.wallet_balance += self.calculate_payout(row, winner)
                else:
                    self.wallet_balance -= self.bet_amount
        return self.wallet_balance

def place_bet_on_index(df, index, bet_amount, bet_team):
    row = df.iloc[index]
    betting_wallet = BettingWallet(bet_amount, bet_team, row['date'], row['home_team'], row['visiting_team'])
    return betting_wallet.place_bet(df)

print(place_bet_on_index(final_df, 5, 1, 1))  # 0 for betting on home team```




-125
-135
101.54074074074074


In [218]:
1+100/182


1.5494505494505495

KeyError: 6